In [1]:
from glob import glob
from datasets import Dataset
import numpy as np
import pandas as pd
from pprint import pprint
import os
import json

### Create training data

In [7]:
ds[0]

{'text': "Việc làm Process Engineering Leader- Trưởng nhóm Kỹ sư Quy trình tại Công Ty TNHH L.S Pack. Địa điểm làm việc: Hồ Chí Minh. Ngành nghề: ['Cơ khí / Ô tô / Tự động hóa', 'Điện / Điện tử / Điện lạnh', 'Bảo trì / Sửa chữa']. Hình thức: ['Nhân viên chính thức']. Lương: 15 Tr - 25 Tr VND. Cấp bậc: Trưởng nhóm / Giám sát. Hết hạn nộp: 30/12/2023. Địa chỉ công ty: Lô A 2-2 Khu công nghiệp Tây Bắc Củ Chi, ấp Bàu Tre 2 - Xã Tân An Hội - Huyện Củ Chi - TP Hồ Chí Minh, Huyện Củ Chi, HCMC. Loại hình hoạt động: None. Quy mô công ty: 25-99. Kinh nghiệm: 1 - 3 năm."}

In [4]:
ds = Dataset.from_json(
    "/Users/phamhoang1408/Desktop/20231/DS/ds_project/crawl/final/final_dataset.jsonl"
)
TEMPLATE = """Việc làm {vi_tri_viec} tại {ten_cong_ty}. Địa điểm làm việc: {dia_diem_lam_viec}. Ngành nghề: {nganh_nghe}. Hình thức: {hinh_thuc}. Lương: {luong}. Cấp bậc: {cap_bac}. Hết hạn nộp: {het_han_nop}. Địa chỉ công ty: {dia_chi_cong_ty}. Loại hình hoạt động: {loai_hinh_hoat_dong}. Quy mô công ty: {quy_mo_cong_ty}. Kinh nghiệm: {min_exp} - {max_exp} năm."""

ds = ds.map(
    lambda x: {
        "text": TEMPLATE.format(
            vi_tri_viec=''x["vi_tri_viec"],
        )
    }
).select_columns(["text"])


In [5]:
ds.to_json('../search_data/data.jsonl', force_ascii=False, orient='records', lines=True)

Creating json from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

4194868

In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import models, losses
from sentence_transformers import (
    SentenceTransformer,
    InputExample,
)
from datetime import datetime


# Training parameters
model_name = "paraphrase-multilingual-mpnet-base-v2"
train_batch_size = 32
num_epochs = 2
max_seq_length = 128

# Save path to store our model
model_save_path = "output/training_stsb_simcse-{}-{}-{}".format(
    model_name, train_batch_size, datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)


# Here we define our SentenceTransformer model
word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


# train_samples is a list of InputExample objects where we pass the same sentence twice to texts, i.e. texts=[sent, sent]
train_samples = []
for line in ds["text"]:
    train_samples.append(InputExample(texts=[line, line]))


# We train our model using the MultipleNegativesRankingLoss
train_dataloader = DataLoader(
    train_samples, shuffle=True, batch_size=train_batch_size, drop_last=True
)
train_loss = losses.MultipleNegativesRankingLoss(model)

warmup_steps = math.ceil(
    len(train_dataloader) * num_epochs * 0.1
)
# Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
    optimizer_params={"lr": 5e-5},
)


model = SentenceTransformer(model_save_path)

In [8]:
str({
    'a': 1,
    'b': 2
})

"{'a': 1, 'b': 2}"